In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

import re

import time

## Mining Team Totals

In [2]:
url = "https://www.basketball-reference.com/leagues/NBA_{}.html"
start_year = 1950
end_year = 2020

dfs = []
for year in range(start_year, end_year+1):
    print(f'Mining year {year}')
    html_content = requests.get(url.format(year)).text
    time.sleep(3) # wait for 2 seconds before making the next request
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", {"id": "totals-team"})
    headers = []
    time.sleep(3) # wait for 2 seconds before making the next request
    for th in table.find("thead").find_all("th"):
        headers.append(th.text)
    data = []
    for tr in table.find("tbody").find_all("tr"):
        row = []
        for td in tr.find_all("td"):
            if td.text == '':
                row.append("Not Found")
            else:
                row.append(td.text)
        data.append(row)

    headers.remove('Rk')
    df = pd.DataFrame(data, columns=headers)
    df["Season"] = f"{year}-{str(year+1)[-2:]}"
    dfs.append(df)

result = pd.concat(dfs, ignore_index=True)
print(result.head())


Mining year 1950
Mining year 1951
Mining year 1952
Mining year 1953
Mining year 1954
Mining year 1955
Mining year 1956
Mining year 1957
Mining year 1958
Mining year 1959
Mining year 1960
Mining year 1961
Mining year 1962
Mining year 1963
Mining year 1964
Mining year 1965
Mining year 1966
Mining year 1967
Mining year 1968
Mining year 1969
Mining year 1970
Mining year 1971
Mining year 1972
Mining year 1973
Mining year 1974
Mining year 1975
Mining year 1976
Mining year 1977
Mining year 1978
Mining year 1979
Mining year 1980
Mining year 1981
Mining year 1982
Mining year 1983
Mining year 1984
Mining year 1985
Mining year 1986
Mining year 1987
Mining year 1988
Mining year 1989
Mining year 1990
Mining year 1991
Mining year 1992
Mining year 1993
Mining year 1994
Mining year 1995
Mining year 1996
Mining year 1997
Mining year 1998
Mining year 1999
Mining year 2000
Mining year 2001
Mining year 2002
Mining year 2003
Mining year 2004
Mining year 2005
Mining year 2006
Mining year 2007
Mining year 20

In [3]:
# define a function to extract the year and increment by 1
def extract_year(season):
    return str(int(re.search(r'\d{4}', season).group()))

# apply the function to the 'Season' column and overwrite it with the new values
result['Season'] = result['Season'].apply(extract_year)

In [4]:
result.T

,0,1,2,3,4,5,6,7,8,9,...,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512
Team,Minneapolis Lakers*,Rochester Royals*,Anderson Packers*,Indianapolis Olympians*,New York Knicks*,Syracuse Nationals*,Boston Celtics,Fort Wayne Pistons*,Chicago Stags*,Tri-Cities Blackhawks*,...,Sacramento Kings,Orlando Magic*,Atlanta Hawks,Minnesota Timberwolves,Detroit Pistons,New York Knicks,Cleveland Cavaliers,Chicago Bulls,Golden State Warriors,Charlotte Hornets
G,68,68,64,64,68,64,68,68,68,64,...,72,73,67,64,66,66,65,65,65,65
MP,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,...,17455,17570,16280,15560,15965,15965,15725,15675,15725,15750
FG,2139,1956,1943,1982,1889,1869,1945,1878,2003,1818,...,2943,2871,2723,2586,2595,2638,2619,2573,2510,2425
FGA,5832,5247,6254,5283,5351,5276,5756,5901,5892,5515,...,6364,6468,6067,5865,5658,5896,5715,5762,5730,5586
FG%,.367,.373,.311,.375,.353,.354,.338,.318,.340,.330,...,.462,.444,.449,.441,.459,.447,.458,.447,.438,.434
3P,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,...,914,807,805,853,791,631,727,793,678,785
3PA,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,...,2511,2354,2416,2540,2157,1872,2070,2282,2032,2231
3P%,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,...,.364,.343,.333,.336,.367,.337,.351,.348,.334,.352
2P,2139,1956,1943,1982,1889,1869,1945,1878,2003,1818,...,2029,2064,1918,1733,1804,2007,1892,1780,1832,1640


In [5]:
# Removing '*' from all team names (they indicate playoff teams)
result['Team'] = result['Team'].str.replace('*', '')

<ipython-input-5-3944150b2f14>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result['Team'] = result['Team'].str.replace('*', '')


In [6]:
result.to_csv('totals_Teams.csv', index=False)

## Mining Teams Standings

In [21]:
url = "https://www.basketball-reference.com/leagues/NBA_{}.html"
start_year = 1980
end_year = 2020

dfs_east = []
dfs_west = []
for year in range(start_year, end_year+1):
    print(f'Mining year {year}')
    html_content = requests.get(url.format(year)).text
    time.sleep(3) # wait for 3 seconds before making the next request
    soup = BeautifulSoup(html_content, "html.parser")
    table_e = soup.find("table", {"id": "divs_standings_E"})
    table_w = soup.find("table", {"id": "divs_standings_W"})
    
    df_e = pd.read_html(str(table_e))[0]
    df_w = pd.read_html(str(table_w))[0]
    
    df_e["Year"] = year
    df_w["Year"] = year
    
    dfs_east.append(df_e)
    dfs_west.append(df_w)

df_east = pd.concat(dfs_east, ignore_index=True)
df_west = pd.concat(dfs_west, ignore_index=True)

Mining year 1980
Mining year 1981
Mining year 1982
Mining year 1983
Mining year 1984
Mining year 1985
Mining year 1986
Mining year 1987
Mining year 1988
Mining year 1989
Mining year 1990
Mining year 1991
Mining year 1992
Mining year 1993
Mining year 1994
Mining year 1995
Mining year 1996
Mining year 1997
Mining year 1998
Mining year 1999
Mining year 2000
Mining year 2001
Mining year 2002
Mining year 2003
Mining year 2004
Mining year 2005
Mining year 2006
Mining year 2007
Mining year 2008
Mining year 2009
Mining year 2010
Mining year 2011
Mining year 2012
Mining year 2013
Mining year 2014
Mining year 2015
Mining year 2016
Mining year 2017
Mining year 2018
Mining year 2019
Mining year 2020


In [32]:
df_east.head(40)

,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Year
1,Boston Celtics,61,21,.744,0,113.5,105.7,7.37,1980
2,Philadelphia 76ers,59,23,.720,2.0,109.1,104.9,4.04,1980
3,Washington Bullets,39,43,.476,22.0,107.0,109.5,-2.27,1980
4,New York Knicks,39,43,.476,22.0,114.0,115.1,-0.96,1980
5,New Jersey Nets,34,48,.415,27.0,108.3,109.5,-0.98,1980
7,Atlanta Hawks,50,32,.610,0,104.5,101.6,2.83,1980
8,Houston Rockets,41,41,.500,9.0,110.8,110.6,0.27,1980
9,San Antonio Spurs,41,41,.500,9.0,119.4,119.7,-0.24,1980
10,Indiana Pacers,37,45,.451,13.0,111.2,111.9,-0.54,1980
11,Cleveland Cavaliers,37,45,.451,13.0,114.1,113.8,0.43,1980


In [23]:
df_east = df_east[~df_east['W/L%'].str.contains("Division")]
df_west = df_west[~df_west['W/L%'].str.contains("Division")]

In [25]:
df_east.rename(columns={'Eastern Conference': 'Team'}, inplace=True)
df_west.rename(columns={'Western Conference': 'Team'}, inplace=True)

In [30]:
df_east.head()

,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Year
1,Boston Celtics,61,21,.744,0,113.5,105.7,7.37,1980
2,Philadelphia 76ers,59,23,.720,2.0,109.1,104.9,4.04,1980
3,Washington Bullets,39,43,.476,22.0,107.0,109.5,-2.27,1980
4,New York Knicks,39,43,.476,22.0,114.0,115.1,-0.96,1980
5,New Jersey Nets,34,48,.415,27.0,108.3,109.5,-0.98,1980


In [31]:
df_west.head()

,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Year
1,Milwaukee Bucks,49,33,.598,0,110.1,106.1,3.57,1980
2,Kansas City Kings,47,35,.573,2.0,108.0,104.9,2.82,1980
3,Denver Nuggets,30,52,.366,19.0,108.3,112.7,-4.22,1980
4,Chicago Bulls,30,52,.366,19.0,107.5,110.2,-2.63,1980
5,Utah Jazz,24,58,.293,25.0,102.4,108.4,-5.71,1980


In [29]:
# Removing '*' from all team names (they indicate playoff teams)
df_east['Team'] = df_east['Team'].str.replace('*', '')
df_west['Team'] = df_west['Team'].str.replace('*', '')

# changing '—' from the column 'GB' to be '0'
df_east['GB'] = df_east['GB'].str.replace('—', '0')
df_west['GB'] = df_west['GB'].str.replace('—', '0')

<ipython-input-29-484d01ae394e>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_east['Team'] = df_east['Team'].str.replace('*', '')
<ipython-input-29-484d01ae394e>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_west['Team'] = df_west['Team'].str.replace('*', '')


In [33]:
dfStandings = pd.concat([df_east, df_west], ignore_index=True)

In [34]:
dfStandings

,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Year
0,Boston Celtics,61,21,.744,0,113.5,105.7,7.37,1980
1,Philadelphia 76ers,59,23,.720,2.0,109.1,104.9,4.04,1980
2,Washington Bullets,39,43,.476,22.0,107.0,109.5,-2.27,1980
3,New York Knicks,39,43,.476,22.0,114.0,115.1,-0.96,1980
4,New Jersey Nets,34,48,.415,27.0,108.3,109.5,-0.98,1980
...,...,...,...,...,...,...,...,...,...
1129,Houston Rockets,44,28,.611,0,117.8,114.8,3.13,2020
1130,Dallas Mavericks,43,32,.573,2.5,117.0,112.1,4.87,2020
1131,Memphis Grizzlies,34,39,.466,10.5,112.6,113.7,-0.91,2020
1132,San Antonio Spurs,32,39,.451,11.5,114.1,115.2,-0.65,2020


In [35]:
dfStandings = dfStandings.sort_values(by=['Year', 'W/L%'], ascending=[True, False])

In [36]:
dfStandings = dfStandings.reset_index(drop=True)

In [37]:
dfStandings.to_csv('Standings.csv', index=False)